# Microtask 2
---
Produce a chart showing the distribution of time-to-close (using the corresponding field in the GrimoireLab enriched index for GitHub issues) for issues already closed, and opened during the last six months.

In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search

import subprocess
import pandas as pd

import matplotlib.pyplot as pd
%matplotlib inline

Assuming we have a running instance of Elasticsearch locally at http://localhost:9200

In [2]:
# elasticsearch instance
es = Elasticsearch('http://localhost:9200', verify_certs=False)

The [grimoirelab-perceval](https://github.com/chaoss/grimoirelab-perceval) repository will be used

In [3]:
org = 'chaoss'
repo = 'grimoirelab-perceval'
token = ''

Run the `p2o.py` subprocess to generate the indices.

In [4]:
subprocess.run(['p2o.py', '--enrich', '--index', 'perceval_github_raw', '--index-enrich', 'perceval_github', '-e', 'http://localhost:9200', '--no_inc', '--debug', 'github', org, repo, '-t', token, '--sleep-for-rate'])

CompletedProcess(args=['p2o.py', '--enrich', '--index', 'perceval_github_raw', '--index-enrich', 'perceval_github', '-e', 'http://localhost:9200', '--no_inc', '--debug', 'github', 'chaoss', 'grimoirelab-perceval', '-t', '1be89fc7151192be77782ef93778861242a48925', '--sleep-for-rate'], returncode=0)

Frame a query for the required information using the `elasticsearch_dsl` module.

In [6]:
s = Search(using=es, index='perceval_github')
# filter by issues
s = s.filter('terms', item_type=['issue'])
# filter issues created in the last six months
s = s.filter('range', created_at={'gte' : 'now-6M'})
# get required fields
s = s.source(['time_to_close_days', 'time_open_days', 'item_type', 'created_at', 'closed_at', 'author_name', 'id_in_repo'])
# sort by creation date
s = s.sort({'created_at': {'order': 'asc'}})
# limit the number of results
s = s[0:1000]

In [12]:
# execute the query
result = s.execute()

In [13]:
result = result.to_dict()['hits']['hits']
result

[{'_id': '6eae638c3522c4c9ee2478686b7bad08876b2476',
  '_index': 'perceval_github',
  '_score': None,
  '_source': {'author_name': 'Santiago Dueñas',
   'closed_at': '2017-09-15T08:45:58Z',
   'created_at': '2017-09-13T09:55:38Z',
   'id_in_repo': '155',
   'item_type': 'issue',
   'time_open_days': 1.95,
   'time_to_close_days': 1.95},
  '_type': 'items',
  'sort': [1505296538000]},
 {'_id': '3d3e02bda92ce73b96056b27052aea783f2d6e43',
  '_index': 'perceval_github',
  '_score': None,
  '_source': {'author_name': 'Santiago Dueñas',
   'closed_at': '2017-10-07T14:41:19Z',
   'created_at': '2017-09-13T10:01:38Z',
   'id_in_repo': '156',
   'item_type': 'issue',
   'time_open_days': 24.19,
   'time_to_close_days': 24.19},
  '_type': 'items',
  'sort': [1505296898000]},
 {'_id': '39b0d3f62a402c3a11997f199af27ddd0e0fab05',
  '_index': 'perceval_github',
  '_score': None,
  '_source': {'author_name': 'Jesus M. Gonzalez-Barahona',
   'closed_at': None,
   'created_at': '2017-09-19T21:52:22Z',
